 * Copyright 2024 Xue_Lexiang
 * Licensed under MIT (https://github.com/xuelx1/LearnQC/LISENCE)

##  Shor algorithm. 3n+2 qubit version.
###  Given a number n, return a non-trivial factor of n. 

### 1.Imports

In [3]:
from funcs import *
from qiskit.circuit import ClassicalRegister
from math import sqrt, isqrt, ceil, gcd, log2
from random import randint
from sympy import isprime
from fractions import Fraction
print("Import completed.")

Import completed.


### 2.Order_Finding Algorithm (Quantum Part)

#### 2.1 build Multiplier Gate

In [4]:
# wait to be completed

def c_amodN(a, power, N):
    n = ceil(log2(N))
    U = QuantumCircuit(n)
    for i in range(power):
        for j in range(n-1):
            U.swap(n-j-2, n-j-1)
        U.x(range(n))
    U = U.to_gate()
    U.name = "%i^%i mod %i" % (a, power, N)
    c_U = U.control()
    return c_U

#### 2.2 build QFT dagger

In [5]:
def qft_dagger(n):
    qc = QuantumCircuit(n)
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    qc.name = "QFT†"
    return qc

#### 2.3 Assemble Order-Finding Circuit

In [6]:
def order_finding(N, a):
    n = ceil(log2(N))
    qc = QuantumCircuit(3*n)
    qc.h(range(2*n))
    qc.x(3*n-1)
    for q in range(2*n):
        qc.append(c_amodN(a, 2**q, N), [q]+[j+2*n for j in range(n)])
    qc.barrier()
    qc.append(qft_dagger(2*n), range(2*n))
    qc = transpile(qc, backend=AerSimulator())
    c = ClassicalRegister(2*n)
    qc.add_register(c)
    for i in range(2*n):
        qc.measure(i, c[i])
    results = execute_qc(qc)
    for key,_ in results.items():
        phase_est = float(int(key, 2)) / 2**(2*n)
        frac = Fraction.from_float(phase_est).limit_denominator(N)
        r = frac.denominator
        if pow(a, r, N) == 1 and r % 2 == 0:
            return r
    return 0
    

### 3. Shor Algorithm (Classical Part)

In [7]:
def shor_factoring(N):
    if N % 2 == 0:
        return 2
    for i in range(2, ceil(log2(N)+1)):
        root = N ** (1 / i) 
        if root == int(root):
            return int(root)
    while True:
        a = randint(2, N-1)
        if gcd(a, N) != 1:
            return gcd(a, N)
        r = order_finding(a, N) # r%2 == 0, otherwise order_finding won't return it.
        if r == 0:
            continue
        if pow(a, int(r/2), N) != -1:
            s1 = gcd(pow(a, int(r/2), N) - 1, N)
            s2 = gcd(pow(a, int(r/2), N) + 1, N)
            if s1 > 1 and s1 < N:
                return s1
            if s2 > 1 and s2 < N:
                return s2


### 4. Factorization

In [8]:
N = 91        

In [9]:
factors = []

def find_inprime(factors):
    for fac_idx in range(len(factors)):
        if not isprime(factors[fac_idx]):
            return fac_idx
    return -1

factors.append(N)
while find_inprime(factors) != -1:
    print(factors)
    fac_idx = find_inprime(factors)
    fac = factors.pop(fac_idx)
    new_fac = shor_factoring(fac)
    factors.append(new_fac)
    factors.append(int(fac/new_fac))
print(factors)


[91]
[13, 7]
